In [1]:
import igraph
import dgl
import torch
import time
import os
import networkx as nx
from datetime import datetime
import pandas as pd
from utils import *
from GRAT import GRAT3

In [2]:
PATH_SAVE_TRAINS = "runs/"

PATH_SAVE_RESULTS = 'results/'
NAME_SAVE_RESULTS = 'FastCover'

PATH_TO_TEST = "../BRKGA/instances/txt/"

FEATURE_TYPE = "1"
HIDDEN_FEATS = [32]*6
input_dim = 32
use_cuda = False
directed_test = False

threshold = 0.5

dt_string = datetime.now().strftime("%m-%d_%H-%M")

## Funciones

### Funciones de difusion

## Evaluando

In [3]:
RUNS_LIST = [run for run in os.listdir(PATH_SAVE_TRAINS) if ".pt" in run]

In [4]:
SEEDS = []
MODELS = []
for run_name in RUNS_LIST:
    SEEDS.append(run_name.split("_")[2])
    MODELS.append(run_name.split("_")[0])

In [5]:
for run_name, model, seed in zip(RUNS_LIST, MODELS, SEEDS):
    print()
    print(f"Evaluation of model: {model}, seed: {seed} in {run_name}")
    print()
    
    if model == 'GRAT':
        net = GRAT3(*HIDDEN_FEATS)
        net.load_state_dict(torch.load(PATH_SAVE_TRAINS+run_name))
    if use_cuda:
        net.cuda()

    Graphs = [graph for graph in os.listdir(PATH_TO_TEST)]
    Graphs = ['graph_football.txt', 'graph_karate.txt', 'graph_dolphins.txt', 'graph_jazz.txt']
    
    graphs = []
    dglgraphs = []
    names = []
    for file in Graphs:
            print(f"Loading {PATH_TO_TEST+file} ...")
            names.append(file.split(".")[0].replace("graph_", ""))

            graph = igraph.Graph().Read_Edgelist("../BRKGA/instances/txt/"+file)
            graphs.append(graph)

            dglgraph = get_rev_dgl(graph, FEATURE_TYPE, input_dim, directed_test, use_cuda)
            dglgraphs.append(dglgraph)

    print()
    print("----- Beginning evaluation -----")
    print()
    records = []
    
    c = 1
    Total = len(names)
    
    for dglgraph, graph, name in zip(dglgraphs, graphs, names):
        start_time = time.time()

        out = net.grat(dglgraph, dglgraph.ndata['feat']).squeeze(1)

        G = graph.to_networkx().to_undirected()

        n = len(G.nodes())

        _, minTargetGRAT = FindMinimumTarget(G, out, threshold)

        final_time = (time.time() - start_time)

        print(f"{c}/{Total} Graph: {name}")
        print(f"Best Target Set length: {minTargetGRAT} out of {n}")
        print(f"Ratio Solution / Graph lentgh: {minTargetGRAT/n:.2f}")
        print()
        records.append({
        "graph": name,
        "model": model,
        "seed": seed,
        "threshold": threshold,
        "n_covered": minTargetGRAT,
        "n": n,
        "coverage": minTargetGRAT/n,
        "t_mean": final_time
        })
    
        pd.DataFrame(records).to_csv(PATH_SAVE_RESULTS + NAME_SAVE_RESULTS +"_" + dt_string + ".csv")
        
        c+=1



Evaluation of model: GRAT, seed: 13 in GRAT_seed_13_10-15_17-40.pt

Loading ../BRKGA/instances/txt/graph_football.txt ...
Loading ../BRKGA/instances/txt/graph_karate.txt ...
Loading ../BRKGA/instances/txt/graph_dolphins.txt ...
Loading ../BRKGA/instances/txt/graph_jazz.txt ...

----- Beginning evaluation -----

1/4 Graph: football
Best Target Set length: 48 out of 115
Ratio Solution / Graph lentgh: 0.42

2/4 Graph: karate
Best Target Set length: 17 out of 62
Ratio Solution / Graph lentgh: 0.27

3/4 Graph: dolphins
Best Target Set length: 12 out of 34
Ratio Solution / Graph lentgh: 0.35

4/4 Graph: jazz
Best Target Set length: 90 out of 198
Ratio Solution / Graph lentgh: 0.45

